In [1]:
from bs4 import BeautifulSoup as BSp
import lxml
#import html5lib alternative to lxml
import requests
import pandas as pd

### 1. Get text from wikipedia website for scraping

In [107]:
#from html file for offline work
with open('Cantest.html') as file:
    soup = BSp(file,'lxml')


#from URL
#URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#source = requests.get(URL).text
#soup = BSp(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<!-- saved from url=(0099)file:///C:/Users/marku/Documents/List%20of%20postal%20codes%20of%20Canada_%20M%20-%20Wikipedia.html -->
<html class="client-js gr__en_wikipedia_org ve-not-available" dir="ltr" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script>
   (function(){var e,t=window.open,r=document.createElement,o=HTMLElement.prototype.appendChild,i=document.createEvent,a={},s=0,u=null,l=window.innerWidth,c=window.innerHeight,f=window.parent,p=window.location!=window.parent.location?document.referrer:document.location.href;function d(){var n=arguments,r=!0,o=null;function i(e){f.postMessage({type:"blockedWindow",args:JSON.stringify(e)},p)}function s(e){var t=e[1];return null!=t&&["_blank","_parent","_self","_top"].indexOf(t)<0?t:null}var u,d,h=null,g=null,m=null;null!=window.event&&(h=window.event.currentTarget,g=window.event.srcElement),null!=g&&g instanceof HTMLElement&&(m=g.closest("a"))&&m.href&&(n[3]=m.href);try{if(n

### 2. Scraping
- find table
- extract all rows
- produce a clean list through spliting and slicing

In [108]:
#isolating table
table = soup.find('table')
rows = table.tbody.text
rows

"\n\nM1A\nNot assigned\nNot assigned\n\n\nM2A\nNot assigned\nNot assigned\n\n\nM3A\nNorth York\nParkwoods\n\n\nM4A\nNorth York\nVictoria Village\n\n\nM5A\nDowntown Toronto\nHarbourfront\n\n\nM5A\nDowntown Toronto\nRegent Park\n\n\nM6A\nNorth York\nLawrence Heights\n\n\nM6A\nNorth York\nLawrence Manor\n\n\nM7A\nQueen's Park\nNot assigned\n\n\nM8A\nNot assigned\nNot assigned\n\n\nM9A\nEtobicoke\nIslington Avenue\n\n\nM1B\nScarborough\nRouge\n\n\nM1B\nScarborough\nMalvern\n\n\nM2B\nNot assigned\nNot assigned\n\n\nM3B\nNorth York\nDon Mills North\n\n\nM4B\nEast York\nWoodbine Gardens\n\n\nM4B\nEast York\nParkview Hill\n\n\nM5B\nDowntown Toronto\nRyerson\n\n\nM5B\nDowntown Toronto\nGarden District\n\n\nM6B\nNorth York\nGlencairn\n\n\nM7B\nNot assigned\nNot assigned\n\n\nM8B\nNot assigned\nNot assigned\n\n\nM9B\nEtobicoke\nCloverdale\n\n\nM9B\nEtobicoke\nIslington\n\n\nM9B\nEtobicoke\nMartin Grove\n\n\nM9B\nEtobicoke\nPrincess Gardens\n\n\nM9B\nEtobicoke\nWest Deane Park\n\n\nM1C\nScarboroug

In [109]:
# create list by spliting the sting
rowList = rows.split('\n')
# clean list
while '' in rowList:
    rowList.remove('')
#remove header
rowList = rowList[3:]
rowList

['M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Harbourfront',
 'M5A',
 'Downtown Toronto',
 'Regent Park',
 'M6A',
 'North York',
 'Lawrence Heights',
 'M6A',
 'North York',
 'Lawrence Manor',
 'M7A',
 "Queen's Park",
 'Not assigned',
 'M8A',
 'Not assigned',
 'Not assigned',
 'M9A',
 'Etobicoke',
 'Islington Avenue',
 'M1B',
 'Scarborough',
 'Rouge',
 'M1B',
 'Scarborough',
 'Malvern',
 'M2B',
 'Not assigned',
 'Not assigned',
 'M3B',
 'North York',
 'Don Mills North',
 'M4B',
 'East York',
 'Woodbine Gardens',
 'M4B',
 'East York',
 'Parkview Hill',
 'M5B',
 'Downtown Toronto',
 'Ryerson',
 'M5B',
 'Downtown Toronto',
 'Garden District',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 'Not assigned',
 'M8B',
 'Not assigned',
 'Not assigned',
 'M9B',
 'Etobicoke',
 'Cloverdale',
 'M9B',
 'Etobicoke',
 'Islington',
 'M9B',
 'Etobicoke',
 'Martin Grove',
 'M9B',
 'Et

### 3. produce dataframe

In [169]:
postcode = rowList[0::3]
Borough = rowList[1::3]
Neighb = rowList[2::3]

dfDict = {'Postcode':postcode,'Borough':Borough,'Neighborhood':Neighb}
df = pd.DataFrame.from_dict(dfDict)
df.head(15)

,Postcode,Borough,Neighborhood
0,M2A,Not assigned,Not assigned
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [170]:
#remove "Not assigned" Boroughs
df = df[df.Borough != "Not assigned"]

#join Neighborhoods by postcode
df = df.groupby(['Postcode', 'Borough'], as_index = False)['Neighborhood'].agg(list)
df['Neighborhood'] = df['Neighborhood'].str.join(', ')

#replace "Not assigned" Neighborhood with Borough
dfTest = df.copy()
dfTest.iloc[:, 1:3] = dfTest.iloc[:, 1:3].where(df['Neighborhood'] != "Not assigned", df['Borough'], axis = 0)
df = dfTest
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)